In [11]:
from pde import CartesianGrid, ScalarField, MemoryStorage, plot_kymograph,PDEBase, Boundaries
import numpy as np
from joblib import load

import json

In [12]:
with open("settings.json",'r') as f:
    params = json.load(f)

In [13]:
class HeatPDE(PDEBase):
    def __init__(self,params):
        self.rho = params["rho"]
        self.rho_l = params["rho_l"]
        self.rho_s = params["rho_s"]
        
        self.cp = cp
        self.cp_s = params["cp_s"]
        self.cp_l = params["cp_l"]
        
        self.k = params["k"]
        self.k_l =  params["k_l"]
        self.k_s = params["k_s"]
        
        self.l_f = params["L_fusion"]
        self.T_l = params["T_L"]
        self.T_s = params["T_S"]
    
    def k_mus(self,temp):
        diff = self.k_s -self.k_l
        k_mus = self.k_l + diff*(temp-self.T_s)
        return k_mus
    
    
    def cp_mus(self,temp):
        diff = self.cp_s -self.cp_l
        cp_mus = self.cp_l + diff*(temp-self.T_s)
        return cp_mus
    
    def rho_mus(self,temp):
        diff = self.rho_s -self.rho_l
        rho_mus = self.rho_l + diff*(temp-self.T_s)
        return rho_mus
    
    def evolution_rate(self, state, t=0):
        T = state.data
        grid = state.grid
        alpha = np.zeros_like(T)
        
        mask_solid = T <= self.T_s
        mask_liquid = T >= self.T_l
        mask_mushy = (T <= self.T_l) & (T > self.T_s)
        
        alpha[mask_solid] = self.k_s /(self.rho_s*self.cp_s)
        alpha[mask_liquid] = self.k_l /(self.rho_l *self.cp_l )
        alpha[mask_mushy] = self.k_mus(T) /(self.rho_mus(T)*self.cp_mus(T) + (self.l_f /(self.T_l-self.T_s)))
        
        lap = state.laplace(bc=[{"value":800.0}, {"value":800.0}])
        rate = alpha * lap
        return ScalarField(grid,rate)
    

In [14]:
length = 15.0e-3
alpha = 1.0e-5  # diffusion coefficient
rho = 1.5
k = 1.34e-3
cp =1190.0
T_l = 890.0
T_s = 750.0
l_f = 389.0e3
grid = CartesianGrid([[0, length]], [50])  # domain from 0 to 1 with 100 points

In [15]:
state = ScalarField(grid, data=np.full(grid.shape, 919.0), with_ghost_cells=False)


In [20]:
# Define Boundary Conditions

pde = HeatPDE(params)




In [17]:
storage = MemoryStorage()
pde.solve(state, t_range=30.0, dt=1e-1, tracker=storage.tracker(0.1), solver="scipy")

plot_kymograph(storage, title="Fixed Dirichlet BCs: 400 -> 400")

ValueError: NumPy boolean array indexing assignment cannot assign 50 input values to the 0 output values where the mask is true